# **Series Temporales**

En este archivo desarrollaremos las siguientes series temporales:
- Time Series Forecasting en Python.
- Modelos estadísticos (AR, ARIMA, SARIMA, Exponential Smoothing).
- Recursive Forecasting (Random Forest, Gradient Boosting Regression).
- Multivariate Forecasting, Ensemble modeling.

Primero importamos todas las librerías que usaremos y las instalamos en caso de ser necesario.

In [1]:
# Importamos las librerías necesarias
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

El próximo paso es cargar los datos limpios.

In [2]:
datos = pd.read_csv('../../data/partidos_limpio.csv')
###datos.head()

,Season,Round,Day,Date,Results,Home,Country (Home),Points (Home),Score (Home),Score (Away),...,MP_away,Starts_away,Gls_away,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away
0,2023-2024,Round of 16,Tue,2024-02-13,A,RB Leipzig,Germany,88.736698,0,1,...,10.0,110.0,20.0,17.0,37.0,20.0,0.0,1.0,18.0,0.0
1,2023-2024,Round of 16,Tue,2024-02-13,A,FC Copenhagen,Denmark,80.431647,1,3,...,10.0,110.0,28.0,20.0,48.0,25.0,3.0,3.0,10.0,0.0
2,2023-2024,Round of 16,Wed,2024-02-14,H,Paris S-G,France,114.334580,2,0,...,8.0,88.0,8.0,5.0,13.0,8.0,0.0,1.0,18.0,0.0
3,2023-2024,Round of 16,Wed,2024-02-14,H,Lazio,Italy,99.943311,1,0,...,10.0,110.0,18.0,14.0,32.0,16.0,2.0,2.0,13.0,1.0
4,2023-2024,Round of 16,Tue,2024-02-20,D,PSV Eindhoven,The Netherlands,98.784903,1,1,...,10.0,110.0,15.0,12.0,27.0,14.0,1.0,1.0,16.0,0.0


### Preparación de datos

Veamos la información general de nuestros datos.

In [3]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          598 non-null    object 
 1   Round           598 non-null    object 
 2   Day             598 non-null    object 
 3   Date            598 non-null    object 
 4   Results         598 non-null    object 
 5   Home            598 non-null    object 
 6   Country (Home)  598 non-null    object 
 7   Points (Home)   598 non-null    float64
 8   Score (Home)    598 non-null    int64  
 9   Score (Away)    598 non-null    int64  
 10  Points (Away)   598 non-null    float64
 11  Country (Away)  598 non-null    object 
 12  Away            598 non-null    object 
 13  Venue           598 non-null    object 
 14  Referee         598 non-null    object 
 15  # Pl_home       538 non-null    float64
 16  Age_home        538 non-null    float64
 17  MP_home         538 non-null    flo

Vemos que todavía tenemos algunas filas nulas. Al limpiar los datos no nos importaba tener algunas filas nulas, pero para hacer la clasterización es muy importante no contar con ningún dato de este tipo.

In [4]:
# Eliminamos las filas que contienen valores nulos
datos = datos.dropna()

# Vemos que se ha hecho el cambio correctamente
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 597
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          538 non-null    object 
 1   Round           538 non-null    object 
 2   Day             538 non-null    object 
 3   Date            538 non-null    object 
 4   Results         538 non-null    object 
 5   Home            538 non-null    object 
 6   Country (Home)  538 non-null    object 
 7   Points (Home)   538 non-null    float64
 8   Score (Home)    538 non-null    int64  
 9   Score (Away)    538 non-null    int64  
 10  Points (Away)   538 non-null    float64
 11  Country (Away)  538 non-null    object 
 12  Away            538 non-null    object 
 13  Venue           538 non-null    object 
 14  Referee         538 non-null    object 
 15  # Pl_home       538 non-null    float64
 16  Age_home        538 non-null    float64
 17  MP_home         538 non-null    flo

Observamos que hay algunas variables categóricas que pasaremos a numéricas para poder incluirlas en nuestros modelos.

In [5]:
# Columnas a modificar
cols = ['Season', 'Round', 'Day', 'Results', 'Home', 'Away', 'Country (Home)', 'Country (Away)', 'Venue', 'Referee']

# Inicializamos el label encoder
label_encoder = LabelEncoder()

# Creamos un diccionario para guardar los mapeos
mapping = {}

# Iteramos sobre las columnas y las transformamos
for col in cols:
    # Concatenamos los valores necesarios
    if col in ['Home', 'Away']:
        if 'Squad' not in mapping:
            name = 'Squad'
            squad = pd.concat([datos['Home'], datos['Away']])
            label_encoder.fit(squad)      
    elif col in ['Country (Home)', 'Country (Away)']:
        if 'Country' not in mapping:
            name = 'Country'
            country = pd.concat([datos['Country (Home)'], datos['Country (Away)']])
            label_encoder.fit(country)
    else:
        name = col
        label_encoder.fit(datos[col])
    
    # Transformamos los valores 
    datos[col] = label_encoder.transform(datos[col])
    
    # Creamos un mapeo de los valores
    mapping[name] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Transformamos la columna 'Date' a datetime
datos['Date'] = pd.to_datetime(datos['Date'])

# Separar la fecha en año, mes y día
datos['Year'] = datos['Date'].dt.year
datos['Month'] = datos['Date'].dt.month
datos['Number Day'] = datos['Date'].dt.day # Lo llamamos 'Number Day' para evitar confusiones con la columna 'Day' que ya existe

# Eliminamos la columna 'Date'
datos.drop('Date', axis=1, inplace=True)

# Verificamos los cambios
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 597
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          538 non-null    int32  
 1   Round           538 non-null    int32  
 2   Day             538 non-null    int32  
 3   Results         538 non-null    int32  
 4   Home            538 non-null    int32  
 5   Country (Home)  538 non-null    int32  
 6   Points (Home)   538 non-null    float64
 7   Score (Home)    538 non-null    int64  
 8   Score (Away)    538 non-null    int64  
 9   Points (Away)   538 non-null    float64
 10  Country (Away)  538 non-null    int32  
 11  Away            538 non-null    int32  
 12  Venue           538 non-null    int32  
 13  Referee         538 non-null    int32  
 14  # Pl_home       538 non-null    float64
 15  Age_home        538 non-null    float64
 16  MP_home         538 non-null    float64
 17  Starts_home     538 non-null    flo

### Modelos

Comenzamos las Series Temporales separando los datos en entrenamiento y prueba.

In [6]:
# Definimos nuestras variables x e y
x = datos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1)
y = datos['Results']

# Dividimos los datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Realizamos los modelos estadísticos de series temporales.

In [7]:
# AR
model_ar = AutoReg(y_train, lags=5)
fit_ar = model_ar.fit()
predictions_ar = fit_ar.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)

# ARIMA
model_arima = ARIMA(y_train, order=(5,1,0))
fit_arima = model_arima.fit()
predictions_arima = fit_arima.forecast(steps=len(y_test))

# SARIMA
model_sarima = SARIMAX(y_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
fit_sarima = model_sarima.fit()
predictions_sarima = fit_sarima.forecast(steps=len(y_test))

# Exponential Smoothing
model_exp = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12)
fit_exp = model_exp.fit()
predictions_exp = fit_exp.forecast(steps=len(y_test))

# Modelos de aprendizaje automático
# Random Forest
model_rf = RandomForestRegressor()
model_rf.fit(x_train, y_train)
predictions_rf = model_rf.predict(x_test)

# Gradient Boosting Regression
model_gb = GradientBoostingRegressor()
model_gb.fit(x_train, y_train)
predictions_gb = model_gb.predict(x_test)

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq

Calculamos su error cuadrático medio para evaluar el rendimiento de cada modelo.

In [8]:
# Evaluación del rendimiento
mse_ar = mean_squared_error(y_test, predictions_ar)
mse_arima = mean_squared_error(y_test, predictions_arima)
mse_sarima = mean_squared_error(y_test, predictions_sarima)
mse_exp = mean_squared_error(y_test, predictions_exp)
mse_rf = mean_squared_error(y_test, predictions_rf)
mse_gb = mean_squared_error(y_test, predictions_gb)

print("Error Cuadrático Medio (AR):", mse_ar)
print("Error Cuadrático Medio (ARIMA):", mse_arima)
print("Error Cuadrático Medio (SARIMA):", mse_sarima)
print("Error Cuadrático Medio (Exponential Smoothing):", mse_exp)
print("Error Cuadrático Medio (Random Forest):", mse_rf)
print("Error Cuadrático Medio (Gradient Boosting):", mse_gb)

Error Cuadrático Medio (AR): 0.7284818234107198
Error Cuadrático Medio (ARIMA): 0.8463910947697247
Error Cuadrático Medio (SARIMA): 0.7650740243280547
Error Cuadrático Medio (Exponential Smoothing): 0.7384435855530385
Error Cuadrático Medio (Random Forest): 0.5211030864197531
Error Cuadrático Medio (Gradient Boosting): 0.5366717552260444


El Error Cuadrático Medio (ECM) cuantifica el promedio de los errores cuadráticos entre las predicciones del modelo y los valores reales. Un ECM más bajo indica que el modelo tiene una mejor capacidad para predecir los valores reales, mientras que un ECM más alto indica que el modelo tiene una peor capacidad predictiva.

Por lo tanto, comparando los valores de los ECMs para cada modelo, podemos concluir que los mejores modelos (es decir, aquellos que predicen mejor) es el Random Forest y el Gradient Boosting.

### Predicciones

Una vez entrenados y evaluados los modelos, pongámoslos a prueba. Vamos a ver qué equipos del siguiente DataFrame pasan a la final.

#### Semifinales

In [9]:
# Definimos una lista llamada 'semis' que contiene datos de partidos de fútbol de las semifinales
semis = [['2023-2024', 'Semi-finals', 'Tue', 'Bayern Munich', 'Germany', 107.882298136646, 114.5545351473923, 'Spain', 'Real Madrid', 'Allianz Arena', 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 2024, 4, 30],
        ['2023-2024', 'Semi-finals', 'Wed', 'Dortmund', 'Germany', 91.17303312629399, 114.33458049886625, 'France', 'Paris S-G', 'Signal Iduna Park', 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 2024, 5, 1],
        ['2023-2024', 'Semi-finals', 'Tue', 'Paris S-G', 'France', 114.33458049886625, 91.17303312629399, 'Germany', 'Dortmund', 'Parc des Princes', 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 2024, 5, 7],
        ['2023-2024', 'Semi-finals', 'Wed', 'Real Madrid', 'Spain', 114.5545351473923, 107.882298136646, 'Germany', 'Bayern Munich', 'Estadio Santiago Bernabéu', 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 2024, 5, 8]]

# Obtenemos las columnas relevantes del DataFrame 'partidos' para usar como nombres de columnas en el DataFrame 'semis'
partidos_cols = datos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1).columns

# Creamos un DataFrame 'semis' a partir de la lista 'semis' con las columnas obtenidas del DataFrame 'partidos'
semis = pd.DataFrame(semis, columns=(partidos_cols))

semis.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,2023-2024,Semi-finals,Tue,Bayern Munich,Germany,107.882298,114.554535,Spain,Real Madrid,Allianz Arena,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,2023-2024,Semi-finals,Wed,Dortmund,Germany,91.173033,114.334580,France,Paris S-G,Signal Iduna Park,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,2023-2024,Semi-finals,Tue,Paris S-G,France,114.334580,91.173033,Germany,Dortmund,Parc des Princes,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,2023-2024,Semi-finals,Wed,Real Madrid,Spain,114.554535,107.882298,Germany,Bayern Munich,Estadio Santiago Bernabéu,...,14,32,16,2,2,13.0,1.0,2024,5,8


Pasamos todas las columnas dategóricas a numéricas.

In [10]:
data = semis.copy()

# Aplicamos mapping a las columnas
for col, col_mapping in mapping.items():
    if col in data.columns:
        data[col] = data[col].map(col_mapping)
    else:
        if col == 'Squad':
            data['Home'] = data['Home'].map(col_mapping)
            data['Away'] = data['Away'].map(col_mapping)
        elif col == 'Country':
            data['Country (Home)'] = data['Country (Home)'].map(col_mapping)
            data['Country (Away)'] = data['Country (Away)'].map(col_mapping)

data.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,18,3,4,7,7,107.882298,114.554535,13,47,1,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,18,3,5,15,7,91.173033,114.334580,6,42,73,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,18,3,4,42,6,114.334580,91.173033,7,15,67,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,18,3,5,47,13,114.554535,107.882298,7,7,30,...,14,32,16,2,2,13.0,1.0,2024,5,8


Hacemos nuestras predicciones y mostramos en pantalla.

In [12]:
# Predicciones
pred_ar_semis = fit_ar.predict(start=len(y_train), end=len(y_train) + len(data) - 1)
pred_arima_semis = fit_arima.forecast(steps=len(data))
pred_sarima_semis = fit_sarima.forecast(steps=len(data))
pred_exp_semis = fit_exp.forecast(steps=len(data))
pred_rf_semis = model_rf.predict(data)
pred_gb_semis = model_gb.predict(data)

# Variables para mostrar los resultados
X_home = semis['Home'].tolist()
X_away = semis['Away'].tolist()
predictions_ar_lst = pred_ar_semis.tolist()
predictions_arima_lst = pred_arima_semis.tolist()
predictions_sarima_lst = pred_sarima_semis.tolist()
predictions_exp_lst = pred_exp_semis.tolist()
predictions_rf_lst = pred_rf_semis.tolist()
predictions_gb_lst = pred_gb_semis.tolist()

# Se crea un DataFrame con los valores obtenidos
res = pd.DataFrame({'Home': X_home, 'Away': X_away, 'AR': predictions_ar_lst, 'ARIMA': predictions_arima_lst, 'SARIMA': predictions_sarima_lst, 'Exponential Smoothing': predictions_exp_lst, 'Random Forest': predictions_rf_lst, 'Gradient Boosting': predictions_gb_lst})

# Mostrar los resultados
res

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an i

,Home,Away,AR,ARIMA,SARIMA,Exponential Smoothing,Random Forest,Gradient Boosting
0,Bayern Munich,Real Madrid,1.205656,1.038450,0.895339,0.999994,1.01,1.291852
1,Dortmund,Paris S-G,1.165122,0.851856,1.248580,1.387091,1.14,1.218042
2,Paris S-G,Dortmund,1.159820,0.974780,0.964461,1.096771,1.66,1.402180
3,Real Madrid,Bayern Munich,1.194593,0.984388,0.926733,1.064492,1.43,1.598405


Obviamente estos valores no nos van a decir el ganador de los partidos. Hagamos una aproximación y cambiemos estos números a su valor correspondiente que encontramos en mapping['Results'].

In [13]:
aprox_ar = [round(valor) for valor in predictions_ar_lst]
aprox_arima = [round(valor) for valor in predictions_arima_lst]
aprox_sarima = [round(valor) for valor in predictions_sarima_lst]
aprox_exp = [round(valor) for valor in predictions_exp_lst]
aprox_rf = [round(valor) for valor in predictions_rf_lst]
aprox_gb = [round(valor) for valor in predictions_gb_lst]

# Mapeamos los valores aproximados
for i in range(len(aprox_ar)):
    aprox_ar[i] = next(key for key, value in mapping['Results'].items() if value == aprox_ar[i])
    aprox_arima[i] = next(key for key, value in mapping['Results'].items() if value == aprox_arima[i])
    aprox_sarima[i] = next(key for key, value in mapping['Results'].items() if value == aprox_sarima[i])
    aprox_exp[i] = next(key for key, value in mapping['Results'].items() if value == aprox_exp[i])
    aprox_rf[i] = next(key for key, value in mapping['Results'].items() if value == aprox_rf[i])
    aprox_gb[i] = next(key for key, value in mapping['Results'].items() if value == aprox_gb[i])

# Modificamos el DataFrame 'res' con los valores aproximados
res['AR'] = aprox_ar
res['ARIMA'] = aprox_arima
res['SARIMA'] = aprox_sarima
res['Exponential Smoothing'] = aprox_exp
res['Random Forest'] = aprox_rf
res['Gradient Boosting'] = aprox_gb

# Mostrar los resultados
res

,Home,Away,AR,ARIMA,SARIMA,Exponential Smoothing,Random Forest,Gradient Boosting
0,Bayern Munich,Real Madrid,D,D,D,D,D,D
1,Dortmund,Paris S-G,D,D,D,D,D,D
2,Paris S-G,Dortmund,D,D,D,D,H,D
3,Real Madrid,Bayern Munich,D,D,D,D,D,H


Vemos que estos modelos predicen empates en nuestros partidos. Por lo tanto, no podremos concluir cuáles son los equipos que pasan a la última ronda del torneo. Lo único que podemos concluir es que según el modelo Random Forest, el PSG pasa a la final y en el modelo de Gradient Boosting el único equipo finalista es el Real Madrid.